# Introduction

This notebook applies a simple location and mode choice model to a PAM population.

The `pam.planner.choice.ChoiceMNL` model allows the user to apply an MNL specification. 

The typical workflow is:

```
choice_model = ChoiceMNL(population, od, zones) # initialize the model and point to the data objects 
choice_model.configure(u, scope) # configure the model by specifying a utility function and the scope of application.
choice_model.apply() # apply the model and update the population with the results.

```

# Dependencies

In [1]:
import pandas as pd
from pam.read import read_matsim
from pam.planner import choice
from pam.operations.cropping import link_population
from pam.planner.od import OD, Labels
import numpy as np
import os
import logging
logging.basicConfig(level=logging.DEBUG)
from prettytable import PrettyTable

In [2]:
%load_ext autoreload
%autoreload 2

# Data

We read an example population, and set the location of all activities to zone `a`:

In [3]:
population = read_matsim(os.path.join('..', 'tests', 'test_data', 'test_matsim_plansv12.xml'))
link_population(population)
for hid, pid, person in population.people():
    for act in person.activities:
        act.location.area = 'a' 

In [4]:
def print_activity_locs(population, act_scope='work'):
    summary = PrettyTable(['pid', 'seq', 'location', 'mode'])
    for hid, pid, person in population.people():
        for seq, act in enumerate(person.plan.activities):
            if (act.act==act_scope) or (act_scope=='all'):
                trmode = act.previous.mode if act.previous is not None else 'NA'
                summary.add_row([pid, seq, act.location.area, trmode])
                
    print(summary)

print('Work locations and travel modes:')
print_activity_locs(population, act_scope='work')

Work locations and travel modes:
+--------+-----+----------+------+
|  pid   | seq | location | mode |
+--------+-----+----------+------+
| chris  |  1  |    a     | car  |
| fatema |  1  |    a     | bike |
|  fred  |  3  |    a     | walk |
| gerry  |  3  |    a     | walk |
|  nick  |  1  |    a     | car  |
+--------+-----+----------+------+


Our `zones` dataset includes destination attraction data, for example the number of jobs or schools in each likely destination zone:

In [5]:
data_zones = pd.DataFrame(
    {
        'zone': ['a', 'b'],
        'jobs': [100, 200],
        'schools': [3, 1]
    }
).set_index('zone')
data_zones

,jobs,schools
zone,,
a,100,3
b,200,1


The `od` objects holds origin-destination data, for example travel time and travel distance between each origin and destination, for each travel mode:

In [6]:
data_od = np.array(
        [[[[20, 30], [40, 45]], [[40, 45], [20, 30]]],
         [[[5,  5], [8,  9]], [[8,  9], [5,  5]]]]
    )

labels = {
    'mode': ['car', 'bus'],
    'vars': ['time', 'distance'],
    'origin_zones': ['a', 'b'],
    'destination_zones': ['a', 'b']
}


od = OD(
    data=data_od,
    labels=labels
)

od

Origin-destination dataset 
--------------------------------------------------
Labels(vars=['time', 'distance'], origin_zones=['a', 'b'], destination_zones=['a', 'b'], mode=['car', 'bus'])
--------------------------------------------------
time - car:
[[20 40]
 [40 20]]
--------------------------------------------------
time - bus:
[[30 45]
 [45 30]]
--------------------------------------------------
distance - car:
[[5 8]
 [8 5]]
--------------------------------------------------
distance - bus:
[[5 9]
 [9 5]]
--------------------------------------------------

# Choice model

In [7]:
planner = choice.ChoiceMNL(population, od, data_zones)

We configure the model by specifying:
* the scope of the model. For example, work activities
* the utility of each alternative

Both options are defined as strings. The stings may comprise mathematical operators, parameters, planner data objects (`od` / `zones`), and/or PAM population objects (`person`/ `act`). 

Parameters can be passed as a list, with each element in the list corresponding to one of the modes in the `od` object.

In [8]:
scope = "act.act=='work'"
asc = [0, -1] # one value for each mode
asc_shift_poor = [-2, 0] # one value for each mode
beta_time = [-0.05, -0.07] # one value for each mode
beta_zones = 0.4
u = f""" \
    {asc} + \
    (np.array({asc_shift_poor}) * (person.attributes['subpopulation']=='poor')) + \
    ({beta_time} * od['time', person.home.area]) + \
    ({beta_zones} * np.log(zones['jobs'].values))
"""

planner.configure(u=u, scope=scope)
print('Utility specification: \n', planner.u)

Utility specification: 
 [0,-1]+(np.array([-2,0])*(person.attributes['subpopulation']=='poor'))+([-0.05,-0.07]*od['time',person.home.area])+(0.4*np.log(zones['jobs'].values))



The `.get_choice_set()` provides with with the utilities of each alternative, as perceived by each agent.

In [12]:
choice_set = planner.get_choice_set()
print('Activities in scope: \n', choice_set.idxs)
print('\nAlternatives: \n', choice_set.choice_labels)
print('\nChoice set utilities: \n', choice_set.u_choices)

Activities in scope: 
 [{'pid': 'chris', 'hid': 'chris', 'seq': 1, 'act': <pam.activity.Activity object at 0x7ff46545a250>}, {'pid': 'fatema', 'hid': 'fatema', 'seq': 1, 'act': <pam.activity.Activity object at 0x7ff46545a910>}, {'pid': 'fred', 'hid': 'fred', 'seq': 3, 'act': <pam.activity.Activity object at 0x7ff4654633a0>}, {'pid': 'gerry', 'hid': 'gerry', 'seq': 3, 'act': <pam.activity.Activity object at 0x7ff465463fa0>}, {'pid': 'nick', 'hid': 'nick', 'seq': 1, 'act': <pam.activity.Activity object at 0x7ff4654671f0>}]

Alternatives: 
 [('a', 'car'), ('a', 'bus'), ('b', 'car'), ('b', 'bus')]

Choice set utilities: 
 [[ 0.84206807 -0.98067305 -0.15793193 -2.03067305]
 [-1.15793193 -0.98067305 -2.15793193 -2.03067305]
 [-1.15793193 -0.98067305 -2.15793193 -2.03067305]
 [-1.15793193 -0.98067305 -2.15793193 -2.03067305]
 [ 0.84206807 -0.98067305 -0.15793193 -2.03067305]]


The `.apply()` method samples from the alternatives, and updates the location and mode of each activity accordingly:

In [10]:
planner.apply()
print('Sampled choices: \n', planner._selections.selections)

INFO:pam.planner.choice:Applying choice model...


Sampled choices: 
 [('a', 'car'), ('b', 'bus'), ('a', 'car'), ('b', 'car'), ('b', 'car')]


The population's activity locations and travel modes have been updated accordingly:

In [11]:
print_activity_locs(planner.population)

+--------+-----+----------+------+
|  pid   | seq | location | mode |
+--------+-----+----------+------+
| chris  |  1  |    a     | car  |
| fatema |  1  |    b     | bus  |
|  fred  |  3  |    a     | car  |
| gerry  |  3  |    b     | car  |
|  nick  |  1  |    b     | car  |
+--------+-----+----------+------+
